In [13]:
import os
import json
import logging 

import openai
from dotenv import load_dotenv
import DuffelManager

load_dotenv("../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [18]:
functions =  [
  {
    "name": "book_best_flight",
    "description": "Will return confirmation for a flight booked given a departure city, arrival city, departure date",
    "parameters": {
      "type": "object",
      "properties": {
        "departure_city": {
          "type": "string",
          "description": "The 3 letter IATA code for the airport in the city of departure e.g. JFK, LHR, CDG"
        },
        "destination_city": {
          "type": "string",
          "description": "The 3 letter IATA code for the airport in the city of arrival e.g. JFK, LHR, CDG"
        },
        "departure_date": {
          "type": "string",
          "description": "The date of departure in the format YYYY-MM-DD"
        },
        "time_of_day": {
          "type": "string",
          "enum": ["morning", "afternoon", "evening", "night"]
        },
        "airline_name": {
          "type": "string",
          "description": "The 2 letter IATA code for the airline the user would like to fly in e.g. DL, UA, AA"
        },
        "cabin_class": {
          "type": "string",
          "enum": ["first", "business", "premium_economy", "economy"]
        },
      },
      "required": ["departure_city", "destination_city", "departure_date"]
    }
  },
  {
    "name": "get_absolute_date",
    "description": "Converts relative dates to absolute dates (only use if you can't convert the query into a date on your own)",
    "parameters": {
      "type": "object",
      "properties": {
        "relative_date": {
          "type": "string",
          "description": "The relative date to convert e.g. tomorrow, next week, next month"
        }
      },
      "required": ["relative_date"]
    }
  }
]

In [24]:
messages = [
  {"role": "system", "content": "You're a friendly flight booking assistant. You will be given a user query. Do your best to parse the user's request and respond accordingly. If the user hasn't given you enough information to make a function call, ask them for clarification"}
]

In [25]:
def call_gpt(chat_history: list[dict]):
  return openai.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=chat_history,
    functions=functions,
    function_call="auto"
  )

def user_interaction(query: str): 
  reason = "function_call"
  messages.append({"role": "user", "content": query})
  
  #allows gpt to continue calling functions until it's ready to return to the user
  while reason == "function_call":
    response = call_gpt(messages)
    logging.info(f"response: {response}")
    reason = response.choices[0].finish_reason
    if reason == "function_call":
      kwargs = json.loads(response.choices[0].message.function_call.arguments)
      function_to_call = getattr(DuffelManager, response.choices[0].message.function_call.name)
      messages.append({"role": "assistant", "content": f"content: {response.choices[0].message.content}\n function_call:{response.choices[0].message.function_call.name}\n"})
      logging.info(f"making {function_to_call.__name__} call with arguments: {kwargs}")
      
      #make the function call and add response to messages
      response_obj = function_to_call(**kwargs)
      if response_obj.success:
        messages.append({"role": "function", "content": str(response_obj.resp), "name": function_to_call.__name__})
      else:
        messages.append({"role": "function", "content": str(response_obj.error), "name": function_to_call.__name__})
    else: # ready to return to user
      messages.append({"role": "assistant", "content": response.choices[0].message.content})
      return messages[-1]["content"]

In [26]:
user_interaction("I'd like to book a flight from London to Adelaide on business class on the first of january in 2024")

2023-11-10 17:10:03,971 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:10:03,973 - root - INFO - response: ChatCompletion(id='chatcmpl-8JTzO8KuOagAvyEQ83BiNQYkaBmLP', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"departure_city":"LHR","destination_city":"ADL","departure_date":"2024-01-01","cabin_class":"business"}', name='book_best_flight'), tool_calls=None))], created=1699654202, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_a24b4d720c', usage=CompletionUsage(completion_tokens=39, prompt_tokens=343, total_tokens=382))
2023-11-10 17:10:03,974 - root - INFO - making book_best_flight call with arguments: {'departure_city': 'LHR', 'destination_city': 'ADL', 'departure_date': '2024-01-01', 'cabin_class': 'business'}
2023-11-10 17:10:03,974 - root - INFO - Getting offer for departure_cit

{'id': 'sto_0000Abg1wIDxw3JAkZ2Rol', 'duration': 'PT6H30M', 'departing_at': '2024-01-03T05:35:00', 'arriving_at': '2024-01-02T23:05:00', 'airport': {'type': 'airport', 'time_zone': 'Australia/Melbourne', 'name': 'Melbourne Airport', 'longitude': 144.842014, 'latitude': -37.671157, 'id': 'arp_mel_au', 'icao_code': 'YMML', 'iata_country_code': 'AU', 'iata_code': 'MEL', 'iata_city_code': 'MEL', 'city_name': 'Melbourne'}}
{'id': 'sto_0000Abg1wIDxw3JAkZ2Rot', 'duration': 'PT6H30M', 'departing_at': '2024-01-03T05:35:00', 'arriving_at': '2024-01-02T23:05:00', 'airport': {'type': 'airport', 'time_zone': 'Australia/Melbourne', 'name': 'Melbourne Airport', 'longitude': 144.842014, 'latitude': -37.671157, 'id': 'arp_mel_au', 'icao_code': 'YMML', 'iata_country_code': 'AU', 'iata_code': 'MEL', 'iata_city_code': 'MEL', 'city_name': 'Melbourne'}}
{'id': 'sto_0000Abg1wIDxw3JAkZ2Rp1', 'duration': 'PT6H30M', 'departing_at': '2024-01-03T05:35:00', 'arriving_at': '2024-01-02T23:05:00', 'airport': {'type':

2023-11-10 17:10:10,383 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:10:10,385 - root - INFO - response: ChatCompletion(id='chatcmpl-8JTzU77dSzREzoorOnMnKWmjXCSQp', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Your flight from London to Adelaide has been booked on Duffel Airways. You'll be flying business class on January 1, 2024. The total for your trip is $3125.67. If you need further assistance or if you want to add any additional services, please let me know!", role='assistant', function_call=None, tool_calls=None))], created=1699654208, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_a24b4d720c', usage=CompletionUsage(completion_tokens=63, prompt_tokens=399, total_tokens=462))


"Your flight from London to Adelaide has been booked on Duffel Airways. You'll be flying business class on January 1, 2024. The total for your trip is $3125.67. If you need further assistance or if you want to add any additional services, please let me know!"

In [27]:
user_interaction("Actually I'd like to make it an economy flight, thats too expensive for me") 

2023-11-10 17:10:15,691 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:10:15,694 - root - INFO - response: ChatCompletion(id='chatcmpl-8JTzaInDA8EWC1BecGdQgQ1p0yi9M', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"departure_city":"LHR","destination_city":"ADL","departure_date":"2024-01-01","cabin_class":"economy"}', name='book_best_flight'), tool_calls=None))], created=1699654214, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_a24b4d720c', usage=CompletionUsage(completion_tokens=40, prompt_tokens=485, total_tokens=525))
2023-11-10 17:10:15,694 - root - INFO - making book_best_flight call with arguments: {'departure_city': 'LHR', 'destination_city': 'ADL', 'departure_date': '2024-01-01', 'cabin_class': 'economy'}
2023-11-10 17:10:15,695 - root - INFO - Getting offer for departure_city:

{'id': 'sto_0000Abg1xLFtGXd0ep53pk', 'duration': 'PT6H30M', 'departing_at': '2024-01-03T05:35:00', 'arriving_at': '2024-01-02T23:05:00', 'airport': {'type': 'airport', 'time_zone': 'Australia/Melbourne', 'name': 'Melbourne Airport', 'longitude': 144.842014, 'latitude': -37.671157, 'id': 'arp_mel_au', 'icao_code': 'YMML', 'iata_country_code': 'AU', 'iata_code': 'MEL', 'iata_city_code': 'MEL', 'city_name': 'Melbourne'}}
{'id': 'sto_0000Abg1xLGbDuCAh1PcwE', 'duration': 'PT6H30M', 'departing_at': '2024-01-03T05:35:00', 'arriving_at': '2024-01-02T23:05:00', 'airport': {'type': 'airport', 'time_zone': 'Australia/Melbourne', 'name': 'Melbourne Airport', 'longitude': 144.842014, 'latitude': -37.671157, 'id': 'arp_mel_au', 'icao_code': 'YMML', 'iata_country_code': 'AU', 'iata_code': 'MEL', 'iata_city_code': 'MEL', 'city_name': 'Melbourne'}}
{'id': 'sto_0000Abg1xLGFFDuafvFLNe', 'duration': 'PT6H30M', 'departing_at': '2024-01-03T05:35:00', 'arriving_at': '2024-01-02T23:05:00', 'airport': {'type':

2023-11-10 17:10:22,437 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:10:22,439 - root - INFO - response: ChatCompletion(id='chatcmpl-8JTzg0bVqOX9AwILEcDLfseUoSoet', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="I've updated your booking to economy class. The flight from London to Adelaide is still on Duffel Airways, with no stops, departing on January 1, 2024. The new price for your trip is $759.19. If everything looks good or if you need further assistance, let me know!", role='assistant', function_call=None, tool_calls=None))], created=1699654220, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_a24b4d720c', usage=CompletionUsage(completion_tokens=65, prompt_tokens=540, total_tokens=605))


"I've updated your booking to economy class. The flight from London to Adelaide is still on Duffel Airways, with no stops, departing on January 1, 2024. The new price for your trip is $759.19. If everything looks good or if you need further assistance, let me know!"

In [ ]:
### things to improve
  # 1. Considered caching requests in case similar requests were made. Might not be worth it since offers change so frequently
  # 2. I haven't made any IATA validators, so airline and flight may not always be correct, but can easily be implemented using the gpt function calling
  # 3. I'd like to expand the success and failure response to include both types of responses (datetime and MyOffer classes) instead of a generic catchall obj

### things to in best offer function
  # 1. Instead of giving up on time of day if there are none at that time, we should try to find the closest time
  # 2. Same idea with cabin class, if there are no offers for the specified cabin class, we should try to find the closest cabin class